Purpose of this script:


dbSNP use the reference strand to annnotate. 


Map  genomic corrdinates with known human variants to other species (Like mouse)

Input: Variant table

In [2]:
import pandas as pd
import os
import gzip
import re
from tqdm import tqdm
%matplotlib inline

In [3]:
pattern='VP=0x\d{4}(\d{2})'
prog = re.compile(pattern)
inDbDir='/data/cellardata/users/btsui/dbsnp/Homo_sapiens/All_20170710.vcf.gz'
outDbDir=inDbDir.replace('.vcf.gz','.f1_byte2_not_00.vcf.gz')
humanDf=pd.read_csv(outDbDir,sep='\t',header=None)

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#!gunzip -c /data/cellardata/users/btsui/dbsnp/Homo_sapiens/All_20170710.vcf.gz |head   -n 20

In [5]:
humanDf.columns=['Chr','Loc','rs','REF','ALT','','','Annot']

In [6]:
## pos strand

In [7]:
#humanDf

### identify regions in human to LiftOVer ( mapping homologous) to mouse

In [11]:
!ls /cellar/users/btsui/Data/ucsc_chains/ | grep Dan

hg38ToDanRer10.over.chain.gz
hg38ToDanRer11.over.chain.gz


In [8]:
inputDir='/cellar/users/btsui/Data/ucsc_chains/'
fileNameInFolderS=pd.Series(os.listdir(inputDir))

In [9]:
fileNameInFolderS[fileNameInFolderS.str.contains('mm10',case=False)]

143    hg38ToMm10.over.chain.gz
dtype: object

In [12]:
chainFiles=fileNameInFolderS[fileNameInFolderS.str.contains('.chain.gz$')]
chainFiles=pd.concat([chainFiles[chainFiles.str.contains('mm10|DanRer',case=False)],
                      chainFiles[~chainFiles.str.contains('mm10|DanRer',case=False)]])

In [13]:
intervalLength=1
refOffset=1
humanDf['Chr_uscs']='chr'+humanDf['Chr'].astype(str)

humanDf['Loc_+1']=humanDf['Loc']+intervalLength-refOffset

humanDf['Loc_-1']=humanDf['Loc']-refOffset
humanDf['Chr_uscs']=humanDf['Chr_uscs'].str.replace('chrMT','chrM')
humanDf['Score']='.'
humanDf['Strand']='+'
humanDf[['Chr_uscs','Loc_-1','Loc_+1','rs','Score','Strand']
       ].to_csv('tmp.human.ucsc.bed',sep='\t',header=None,index=None)
#humanDf[['Chr','Loc_-1','Loc_+1','rs']].to_csv('tmp.human.ensembl.bed',sep='\t',header=None,index=None)

In [14]:
#!head tmp.human.ucsc.bed

### make chains

In [ ]:
for chainFile in tqdm(chainFiles):
    outDir='/cellar/users/btsui/Data/ucsc_chains_human_homo_snps/{}'.format(chainFile+'.human_homo.bed')   
    unmappedOutDir='/cellar/users/btsui/Data/ucsc_chains_human_homo_snps/{}'.format(chainFile+'.human_homo.unmapped.bed')   
    liftOverCmd="liftOver {} {} {} {}".format('tmp.human.ucsc.bed',
                                                     inputDir+chainFile,
                                                     outDir,unmappedOutDir)
    print (os.system(liftOverCmd))

  1%|          | 1/157 [00:01<03:20,  1.29s/it]

0


  1%|▏         | 2/157 [00:03<04:30,  1.74s/it]

0


  2%|▏         | 3/157 [08:08<6:58:07, 162.91s/it]

...

0


  7%|▋         | 11/157 [29:17<6:28:48, 159.78s/it]

0


  8%|▊         | 12/157 [46:33<9:22:34, 232.79s/it]

0


In [ ]:
#!mkdir /cellar/users/btsui/Data/ucsc_chains_human_homo_snps/

In [ ]:
asdasd

In [ ]:
#os.system(liftOverCmd)

###

# Scratch

### check SNP strand infos

In [11]:
humanDf[['Chr','Loc_-1','Loc','rs']].to_csv('tmp.human.ensembl.bed',sep='\t',header=None,index=None)

!bedtools getfasta -fi /cellar/users/btsui/Data/ensembl/clean/Homo_sapiens.fa -bed tmp.human.ensembl.bed -fo tmp.human.ucsc.bed.fasta

In [12]:
tmpS=pd.read_csv('tmp.human.ucsc.bed.fasta',header=None)[0]

In [13]:
extractedDf=pd.DataFrame({'Loc':tmpS[(tmpS.index%2)==0].values,
              'Nucleotide':tmpS[(tmpS.index%2)==1].values})

In [14]:
extractedDf['Chr']=extractedDf.Loc.str.extract('(\w+):')

In [15]:
extractedDf['Site']=extractedDf.Loc.str.extract('\w+:\d+-(\d+)')

In [16]:
extractedDf['Chr-Site']=extractedDf['Chr']+'-'+extractedDf['Site'].astype(str)

In [17]:
chrToPosN=extractedDf.set_index(['Chr-Site'])['Nucleotide']

In [18]:
chrToPosN=chrToPosN.groupby(chrToPosN.index).first()

In [19]:
humanDf.head()

Chr     Loc          rs REF ALT        \
0   1   14727   rs1045587   G   A  .  .   
1   1  630825   rs9783068   T   C  .  .   
2   1  630833   rs9701099   C   T  .  .   
3   1  817186   rs3094315   G   A  .  .   
4   1  833068  rs12562034   G   A  .  .   

                                               Annot Chr_uscs  Loc_+1  Loc_-1  
0  RS=1045587;RSPOS=14727;RV;dbSNPBuildID=117;SSR...     chr1   14727   14726  
1  RS=9783068;RSPOS=630825;dbSNPBuildID=119;SSR=1...     chr1  630825  630824  
2  RS=9701099;RSPOS=630833;dbSNPBuildID=119;SSR=1...     chr1  630833  630832  
3  RS=3094315;RSPOS=817186;RV;dbSNPBuildID=103;SS...     chr1  817186  817185  
4  RS=12562034;RSPOS=833068;dbSNPBuildID=120;SSR=...     chr1  833068  833067

In [20]:
humanDf['Chr-Site']=humanDf['Chr'].astype(str)+'-'+humanDf['Loc'].astype(str)#chrToPosN[extractedDf['Chr-Site']].values

In [21]:
#posStrandDf=humanDf[~humanDf.Annot.str.contains(';RV;')]

In [22]:
humanDf['Pos_N']=chrToPosN.loc[humanDf['Chr-Site']].values

In [28]:
humanDf.groupby(['Pos_N','REF']).size().sort_values()

Pos_N  REF                     
G      GCCTCCAGCCCCAGCT                1
       GTGTAAACTCAGAAA                 1
       GTGTACCCTT                      1
       GTGTCAT                         1
       GTGTGCCA                        1
       GTGTGGGGCGGAGACTT               1
       GTGTGT                          1
       GTGTGTCAGC                      1
       GTGTTCCAAGGGGACAGTAGCCCC        1
       GTTA                            1
       GTTAAA                          1
       GTTAAATCCA                      1
       GTTAAGGAA                       1
       GTTACAAT                        1
       GTTACAATTTACTGGCA               1
       GTTACAGGAACT                    1
       GTTACC                          1
       GTTAGATCC                       1
       GTTCTCCTCATTGAAAAAGA            1
       GTTCTC                          1
       GTTCTACAACAAGAGCGAGGAT          1
       GTTCCT                          1
       GTTCCATA                        1
       GTTCAGCT          

In [26]:
#tmpS.str.extract('(\d+):\d+-(\d+)',expand=True)

In [27]:
#posStrandDf

In [ ]:
tmpS

### map the human variants to mouse

In [ ]:
#!head tmp.mouse.bed

In [ ]:
## remove 1000 when not testing
liftOverBedDf=pd.read_csv('tmp.mouse.bed',sep='\t',nrows=100000,header=None) 

liftOverBedDf.columns=['Chr_uscs','Start','End','rs']

###slice out the region for mouse with name

liftOverBedDf['Chr_ensmbl']=liftOverBedDf['Chr_uscs'].str.replace('^chr','')
liftOverBedDf['Start']=liftOverBedDf['Start']
liftOverBedDf['End']=liftOverBedDf['End']
liftOverBedDf[['Chr_ensmbl','Start','End','rs']].to_csv('tmp.specie.ensembl.bed',sep='\t',header=None,index=None)

In [ ]:
specieFastaDir='/cellar/users/btsui/Data/ensembl/release/fasta/Mus_musculus.GRCm38.dna_rm.toplevel.fa'

In [ ]:
bedInterCmd="bedtools getfasta -name -fi {} -bed tmp.specie.ensembl.bed > liftOver.fasta".format(specieFastaDir)

In [ ]:
#bedInterCmdHuman="bedtools getfasta -name -fi {} -bed tmp.human.ucsc.bed > liftOver.human.fasta".format(humanFastaDir)

In [ ]:
#!head liftOver.human.fasta

In [ ]:
#!bedtools getfasta -name -fi /cellar/users/btsui/Data/ensembl/release/fasta/Homo_sapiens.GRCh38.dna_rm.toplevel.fa -bed tmp.human.ucsc.bed > liftOver.human.fasta

In [ ]:
os.system(bedInterCmdHuman)

In [ ]:
!bedtools getfasta -name -fi /cellar/users/btsui/Data/ensembl/release/fasta/Mus_musculus.GRCm38.dna_rm.toplevel.fa -bed tmp.specie.ensembl.bed > liftOver.fasta

In [ ]:
os.system(bedInterCmd)

In [ ]:
#pd.read_csv('liftOver.human.fasta')

In [ ]:
myFastaS=pd.read_csv( 'liftOver.fasta',header=None)[0]

rsToSeqDf=pd.DataFrame({'rs':myFastaS[(myFastaS.index%2)==0].str.replace('>','').values,
                        'Seq':myFastaS[(myFastaS.index%2)==1].values})

rsToSeqSpecie=rsToSeqDf.set_index('rs')['Seq']

revS=pd.Series({'A':'T','T':'A','C':'G','G':'C'})


rsToSeqSpecie_corrected=pd.Series(data=revS[rsToSeqSpecie].values,index=rsToSeqSpecie.index)

humanDf['ref_sepcie']=rsToSeqSpecie_corrected[humanDf['rs']].values

In [ ]:
#withValidDf.Chr

In [ ]:
withValidDf=humanDf[humanDf['ref_sepcie'].isin(['A','C','G','T'])&(humanDf.Chr==3)]

In [ ]:
(withValidDf['ref_sepcie']==withValidDf['REF']).mean()

In [ ]:
#withValidDf#.mean()

In [ ]:
#(withValidDf['REF']==withValidDf['ref_sepcie'])[withValidDf['Loc']<100000000].astype(int).reset_index().plot(x='index',y=0,kind='scatter',alpha=0.5)

In [ ]:
#withValidDf.Chr

In [ ]:
#withValidDf[]

In [ ]:
(withValidDf['REF']==withValidDf['ref_sepcie'])[withValidDf['Loc']<100000000].astype(int).reset_index().plot(x='Loc',y=0,kind='scatter',alpha=0.5)

In [ ]:
#(withValidDf['REF']==withValidDf['specie_ref_seq']).mean()

In [ ]:
withValidDf[(withValidDf['REF']==withValidDf['specie_ref_seq'])]

In [ ]:
#withValidDf

In [ ]:
revS

In [ ]:
myDict={}
for i  in range(0,50,1):
    myDict[i]={'rev':(withValidDf['specie_ref_seq'].str[i]==revS[withValidDf['REF']].values).mean(),
               'pos':(withValidDf['specie_ref_seq'].str[i]==withValidDf['REF'].values).mean()
              }

In [ ]:
withValidDf['rev_ref']=revS[withValidDf['REF']].values

In [ ]:
#withValidDf[withValidDf.specie_ref_seq.str[0]!=revS[withValidDf['REF']].values]

In [ ]:
#withValidDf

In [ ]:
pd.DataFrame(myDict).T#.plot()

In [ ]:
i=25
#pd.Series(myDict)

In [ ]:
tmpDf=pd.DataFrame( {'specie':withValidDf['specie_ref_seq'].str[i].values,'ref':withValidDf['REF'].values})
tmpDf['rev_ref']=revS[tmpDf['ref']].values

In [ ]:
(tmpDf['rev_ref']==tmpDf['specie']).mean()

In [ ]:
pd.Series(myDict
         ).plot()

In [ ]:
"""myDict={}
for i  in range(0,100,1):
    myDict[i]=(withValidDf['specie_ref_seq'].str[i]).value_counts()"""

In [ ]:
#withValidDf['specie_ref_seq'].str.len()

In [ ]:
pd.DataFrame(myDict).T.plot()

In [ ]:
pd.Series(myDict)#.plot()

In [ ]:
!head tmp.human.ensembl.bed

In [ ]:
#!bedtools getfasta -fi /cellar/users/btsui/Data/ensembl/release/fasta/Mus_musculus.GRCm38.dna_rm.toplevel.fa -bed tmp.human.ensembl.bed > liftOver.bed

In [ ]:
#!getFastaFromBed -fi 

In [ ]:
#getFastaFromBed [OPTIONS] -fi <input FASTA> -bed <BED/GFF/VCF>


In [ ]:
## get the fasta 

In [ ]:
#liftOverBedDf

In [ ]:
!wc -l tmp.mouse.bed

In [ ]:
!wc -l unMapped

In [ ]:
#!liftOver

### check in fasta to make sure ref base is correct